In [1]:
!pip install ctransformers accelerate
!pip install sacrebleu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 35.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [55]:
import pandas as pd
import sacrebleu
import time

In [56]:
en_file_path = "/content/English (openchat).txt"
pr_file_path = "/content/Persian (openchat).txt"

with open(en_file_path, "r") as file:
    english = [line.strip() for line in file]

with open(pr_file_path, "r") as file:
    persian = [line.strip() for line in file]

In [57]:
en_sentences_10 = [
    '"in the heat and sands of India, the mud and swamp of Africa, the purlieus of London, wherever in short the climate or the devil tempts men to fall from the true belief which is her own is even now engaged in dashing down shrines, smashing idols, and setting up in their place her own stern countenance."',
    'Conversion is her name',
    '"she feasts on the wills of the weakly, loving to impress, to impose, adoring her own features stamped on the face of the populace."',
    'At Hyde Park Corner on a tub she stands preaching;',
    '"shrouds herself in white and walks penitentially disguised as brotherly love through factories and parliaments; offers help, but desires power;"',
    '"smites out of her way roughly the dissentient, or dissatisfied; bestows her blessing on those who, looking upward, catch submissively from her eyes the light of their own."',
    '"This lady too Rezia Warren Smith divined it had her dwelling in Sir William\'s heart, though concealed, as she mostly is, under some plausible disguise; some venerable name; love, duty, self sacrifice."',
    'How he would work',
    '"how toil to raise funds, propagate reforms, initiate institutions!"',
    '"But conversion, fastidious Goddess, loves blood better than brick, and feasts most subtly on the human will."'
    ]

In [45]:
pr_sentences_10 = [
    "در گرما و ماسه هایس هندوستان، گل و باتلاق آفریقا، حواشی لندن، هر جا کهخلاصه اقلیم یا شیطان انسانتن‌ها را وسوسه می‌کند تا از صراط راستینی که از آن اوست بلغزند حتی در همین لحظه مشغول پایین کشیدن محراب‌ها، خرد کردن بت‌ها و بر پا کردن چهرة عبوس خود به جای آن‌هاست.",
    "نامش همکیش گردانی است",
    "از ارادة ضعیفان سور خویش فراهم می‌آورد، عاشق تاثیر گذاشتن است، تحمیل کردن، ستایش وجنات خویش که به صورت عوام الناس مهر شده است.",
    "در هاید پارک کرنر بر گلدانی ایستاده موعظه می‌کند؛",
    "پیچیده در دایی سفید به نشان توبه با لباس مبدل عشق برادرانه در کارخانه‌ها و مجالس قانونگذاری راه می‌رود؛ پیشنهاد کمک می‌کند، اما طالب قدرت است.",
    "مخالفان، یا ناراضیان را با خشونت از سر راه خود کنار میزند؛ دعای خیر خود را نثار آنان می‌کند که رو به بالاکرده، مطیعانه فروغ چشمان خود را از چشمان او می‌گیرند.",
    "این بانو هم رتزیا وارن اسمیت به فراست در یافت د قلب سر ویلیام جایداشت هر چند پنهان بود چنان که اغلب هست در لباس مبدلی موجه ناامی والا عشق وظیفه از خود گذشتگی.",
    "اماده که چه کارها بکند",
    "چه زحمت‌ها که بکشد تا منابع مالی را تامین کند اصطلاحات را ترویج کند نهادهایی به راه اندازد.",
    "اما همکیش گردانی ایزد بانوی سختگیر خون را بیش از خشت دوست دارد و به ظریف‌ترین وجهی اراده‌ی انسان را سفره‌ی سور خود می‌کند."
]

## English Prompt - English to Persian

### Zeroshot

In [7]:
from ctransformers import AutoModelForCausalLM

# Load mô hình lượng tử hóa OpenChat-3.5 từ TheBloke
model = AutoModelForCausalLM.from_pretrained(
    "TheBloke/openchat-3.5-0106-GGUF",
    model_file="openchat-3.5-0106.Q4_K_M.gguf",
    model_type="llama",
    gpu_layers=20,
    context_length=2048
)

# Định dạng prompt kiểu hội thoại
def build_prompt(user_input):
    return f"<|user|>\n{user_input}\n<|assistant|>\n"

# Hàm sinh văn bản từ mô hình
def get_completion(prompt):
    return model(prompt).strip()

# Hàm dịch từng câu và lưu ra file
def translate_and_save(sentences, source_language, target_language, output_file):
    with open(output_file, 'w', encoding='utf-8') as f:
        for sentence in sentences:
            user_prompt = f"Translate the following {source_language} sentence into {target_language}:\n\n\"{sentence}\""
            prompt = build_prompt(user_prompt)

            try:
                response = get_completion(prompt)
            except Exception as e:
                response = "[Translation Failed]"
                print(f"Error: {e}")

            f.write(response + '\n')
            time.sleep(20)  # delay tránh tràn bộ nhớ

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

openchat-3.5-0106.Q4_K_M.gguf:   0%|          | 0.00/4.37G [00:00<?, ?B/s]

In [8]:
output_path = "/content/OpenChat-EN-PER-0shot (openchat).txt"
translate_and_save(
    en_sentences_10,
    source_language = "English",
    target_language = "Persian",
    output_file = output_path)

In [10]:
enenpr0_file_path = "/content/OpenChat-EN-PER-0shot (openchat).txt"

with open(enenpr0_file_path, "r") as file:
    enenpr0 = [line.strip() for line in file]

bleu = sacrebleu.corpus_bleu(enenpr0, [persian])

print(bleu)

BLEU = 0.24 10.2/1.2/0.2/0.1 (BP = 0.313 ratio = 0.463 hyp_len = 547 ref_len = 1182)


### Oneshot

In [15]:
# Định dạng prompt kiểu hội thoại
def build_prompt(user_input):
    return f"<|user|>\n{user_input}\n<|assistant|>\n"

# Hàm sinh văn bản từ mô hình
def get_completion(prompt):
    return model(prompt).strip()

def translate_and_save(sentences, source_language, target_language, output_file):
    # Prompt one-shot với ví dụ mẫu
    prompt_intro = (
        f"In this task, which is Machine Translation (MT), you will be presented with a sentence in {source_language}. "
        f"You should translate it to {target_language} in the most appropriate way.\n\n"
        f"Here is an example for this task, which is translating sentences from {source_language} to {target_language}:\n"
        f"'This is a sample sentence that we are going to translate it.' -> 'این یک جمله‌ی نمونه است که می‌خواهیم آن را ترجمه کنیم.'"
    )

    with open(output_file, 'w', encoding='utf-8') as f:
        for sentence in sentences:
            user_prompt = f"{prompt_intro}\n\n{source_language} to {target_language}: {sentence}"
            prompt = build_prompt(user_prompt)

            try:
                response = get_completion(prompt)
            except Exception as e:
                response = "[Translation Failed]"
                print(f"Error: {e}")

            f.write(response.strip() + '\n')
            time.sleep(20)

In [16]:
translate_and_save(
    en_sentences_10,
    source_language="English",
    target_language="Persian",
    output_file="/content/ENPrompt-EN-PER-1shot (openchat).txt"
)

In [17]:
enenpr1_file_path = "/content/ENPrompt-EN-PER-1shot (openchat).txt"

with open(enenpr1_file_path, "r") as file:
    enenpr1 = [line.strip() for line in file]

bleu = sacrebleu.corpus_bleu(enenpr1, [persian])

print(bleu)

BLEU = 0.08 7.5/0.5/0.1/0.1 (BP = 0.173 ratio = 0.363 hyp_len = 429 ref_len = 1182)


### Fewshot

In [22]:
# Định dạng prompt kiểu hội thoại
def build_prompt(user_input):
    return f"<|user|>\n{user_input}\n<|assistant|>\n"

# Hàm sinh văn bản từ mô hình
def get_completion(prompt):
    return model(prompt).strip()

def translate_and_save(sentences, source_language, target_language, output_file):
    prompt_intro = (
        f"In this task, which is Machine Translation (MT), you will be presented with a sentence in {source_language}. "
        f"You should translate it to {target_language} in the most appropriate way.\n\n"

        f"Here are some examples for this task:\n"
        f"'I love programming.' -> 'من برنامه‌نویسی را دوست دارم.'\n"
        f"'Cooking a delicious meal requires a combination of ingredients and techniques.' -> 'پخت یک وعده غذای خوشمزه نیازمند ترکیبی از مواد اولیه و تکنیک‌هاست.'\n"
        f"'In the realm of artificial intelligence, neural networks play a pivotal role.' -> 'در دنیای هوش مصنوعی، شبکه‌های عصبی نقشی کلیدی ایفا می‌کنند.'"
    )

    with open(output_file, 'w', encoding='utf-8') as f:
        for sentence in sentences:
            prompt = f"{prompt_intro}\n\n{source_language} to {target_language}: {sentence}"
            response = get_completion(prompt)
            f.write(response.strip() + '\n')
            time.sleep(24)  # Tránh nóng máy

In [23]:
translate_and_save(
    en_sentences_10,
    source_language="English",
    target_language="Persian",
    output_file="/content/ENPrompt-EN-PER-3shot (openchat).txt"
    )

In [24]:
enenpr3_file_path = "/content/ENPrompt-EN-PER-3shot (openchat).txt"

with open(enenpr3_file_path, "r") as file:
    enenpr3 = [line.strip() for line in file]

bleu = sacrebleu.corpus_bleu(enenpr3, [persian])

print(bleu)

BLEU = 0.32 8.8/0.8/0.1/0.1 (BP = 0.651 ratio = 0.700 hyp_len = 827 ref_len = 1182)


## English Prompt - Persian to English

### Zeroshot

In [30]:
# Định dạng prompt kiểu hội thoại
def build_prompt(user_input):
    return f"<|user|>\n{user_input}\n<|assistant|>\n"

# Hàm sinh văn bản từ mô hình
def get_completion(prompt):
    return model(prompt).strip()

def translate_and_save(sentences, source_language, target_language, output_file):
    # Prompt for machine translation task
    prompt_intro = (
        f"In this task, which is Machine Translation (MT), you will be presented with a sentence in the {source_language} (Farsi) language."
        f"You should translate it to {target_language} in the most appropriate way."
    )
    # Get the translation for each sentence and save it to a file
    with open(output_file, 'w', encoding='utf-8') as f:
        for sentence in sentences:
            prompt = f"{prompt_intro}\n{source_language} to {target_language}: {sentence}"

            # Get the translation for the current sentence
            response = get_completion(prompt)

            # Extract and save the translated sentence
            translated_sentence = response.strip()
            f.write(translated_sentence + '\n')

            # Add a delay of 20 seconds between requests
            time.sleep(21)

In [31]:
translate_and_save(
    pr_sentences_10,
    source_language="Persian",
    target_language="English",
    output_file="/content/ENPrompt-PER-EN-0shot (openchat).txt"
)

In [32]:
enpren1_file_path = "/content/ENPrompt-PER-EN-0shot (openchat).txt"

with open(enpren1_file_path, "r") as file:
    enpren0 = [line.strip() for line in file]

bleu = sacrebleu.corpus_bleu(enpren0, [english])

print(bleu)

BLEU = 1.68 12.1/1.7/0.9/0.7 (BP = 0.889 ratio = 0.895 hyp_len = 1027 ref_len = 1148)


### Oneshot

In [40]:
# Định dạng prompt kiểu hội thoại
def build_prompt(user_input):
    return f"<|user|>\n{user_input}\n<|assistant|>\n"

# Hàm sinh văn bản từ mô hình
def get_completion(prompt):
    return model(prompt).strip()

def translate_and_save(sentences, source_language, target_language, output_file):
    # Prompt for machine translation task
    prompt_intro = (
        f"In this task, which is Machine Translation (MT), you will be presented with a sentence in {source_language}"
        f"You should translate it to {target_language} in the most appropriate way."

        f"Here is an example for this task, which is translating sentecnes from {source_language} to {target_language}:"
        "'این یک جمله‌ی نمونه است که می‌خواهیم آن را ترجمه کنیم' -> 'This is a sample sentence that we are going to translate it.'"
    )

    # Get the translation for each sentence and save it to a file
    with open(output_file, 'w', encoding='utf-8') as f:
        for sentence in sentences:
            prompt = f"{prompt_intro}\n{source_language} to {target_language}: {sentence}"

            # Get the translation for the current sentence
            response = get_completion(prompt)

            # Extract and save the translated sentence
            translated_sentence = response.strip()
            f.write(translated_sentence + '\n')

            # Add a delay of 20 seconds between requests
            time.sleep(21)

In [41]:
translate_and_save(
    pr_sentences_10,
    source_language="Persian",
    target_language="English",
    output_file="/content/ENPrompt-PER-EN-1shot (openchat).txt"
)

In [42]:
enpren3_file_path = "/content/ENPrompt-PER-EN-1shot (openchat).txt"

with open(enpren3_file_path, "r") as file:
    enpren0 = [line.strip() for line in file]

bleu = sacrebleu.corpus_bleu(enpren0, [english])

print(bleu)

BLEU = 1.79 14.0/3.9/2.0/1.2 (BP = 0.530 ratio = 0.611 hyp_len = 494 ref_len = 808)


### Fewshot

In [46]:
# Định dạng prompt kiểu hội thoại
def build_prompt(user_input):
    return f"<|user|>\n{user_input}\n<|assistant|>\n"

# Hàm sinh văn bản từ mô hình
def get_completion(prompt):
    return model(prompt).strip()

def translate_and_save(sentences, source_language, target_language, output_file):
    # Prompt for machine translation task
    prompt_intro = (
        f"In this task, which is Machine Translation (MT), you will be presented with a sentence in {source_language}"
        f"You should translate it to {target_language} in the most appropriate way."

        f"Here is an example for this task, which is translating sentecnes from {source_language} to {target_language}:"
        "'من عاشق برنامه‌نویسی هستم.' -> 'I love programming.'"
        "'پخت یک وعده غذای خوشمزه نیازمند ترکیبی از مواد اولیه و تکنیک‌هاست.' -> 'Cooking a delicious meal requires a combination of ingredients and techniques.'"
        "'در دنیای هوش مصنوعی، شبکه‌های عصبی نقشی کلیدی ایفا می‌کنند.' -> 'In the realm of artificial intelligence, neural networks play a pivotal role.'"
    )

    # Get the translation for each sentence and save it to a file
    with open(output_file, 'w', encoding='utf-8') as f:
        for sentence in sentences:
            prompt = f"{prompt_intro}\n{source_language} to {target_language}: {sentence}"

            # Get the translation for the current sentence
            response = get_completion(prompt)

            # Extract and save the translated sentence
            translated_sentence = response.strip()
            f.write(translated_sentence + '\n')

            # Add a delay of 20 seconds between requests
            time.sleep(20)

In [47]:
translate_and_save(
    pr_sentences_10,
    source_language="Persian",
    target_language="English",
    output_file="/content/ENPrompt-PER-EN-3shot (openchat).txt"
)

In [48]:
enpren0_file_path = "/content/ENPrompt-PER-EN-3shot (openchat).txt"

with open(enpren0_file_path, "r") as file:
    enpren0 = [line.strip() for line in file]

bleu = sacrebleu.corpus_bleu(enpren0, [english])

print(bleu)

BLEU = 0.13 6.3/0.2/0.1/0.1 (BP = 0.410 ratio = 0.529 hyp_len = 495 ref_len = 936)


## Persian Prompt - English to Persian

### Zeroshot

In [52]:
# Định dạng prompt kiểu hội thoại
def build_prompt(user_input):
    return f"<|user|>\n{user_input}\n<|assistant|>\n"

# Hàm sinh văn bản từ mô hình
def get_completion(prompt):
    return model(prompt).strip()

def translate_and_save(sentences, source_language, target_language, output_file):
    # Prompt for machine translation task
    prompt_intro = (
        "در ادامه‌، وظیفه‌ (تکلیفی) برای شما درنظر گرفته شده است، که مربوط به  ترجمه ماشینی است. "
        "به این منظور جمله‌ای به زبان انگلیسی به شما نشان داده می‌شود. "
        "در نتیجه، شما باید آن جمله را به شکل مناسب و دقیق به زبان فارسی ترجمه کنید."
    )

    # Get the translation for each sentence and save it to a file
    with open(output_file, 'w', encoding='utf-8') as f:
        for sentence in sentences:
            prompt = f"{prompt_intro}\n\nجمله زیر به زبان فارسی است و باید به زبان انگلیسی ترجمه شود:\n {sentence}"

            # Get the translation for the current sentence
            response = get_completion(prompt)

            # Extract and save the translated sentence
            translated_sentence = response.strip()
            f.write(translated_sentence + '\n')

            # Add a delay of 20 seconds between requests
            time.sleep(20)

In [53]:
translate_and_save(
    en_sentences_10,
    source_language="English",
    target_language="Persian",
    output_file="/content/PERPrompt-EN-PER-0shot (openchat).txt"
)

In [54]:
prener0_file_path = "/content/PERPrompt-EN-PER-0shot (openchat).txt"

with open(prener0_file_path, "r") as file:
    prener0 = [line.strip() for line in file]

bleu = sacrebleu.corpus_bleu(prener0, [persian])

print(bleu)

BLEU = 0.12 9.6/0.2/0.1/0.0 (BP = 0.406 ratio = 0.526 hyp_len = 622 ref_len = 1182)


### Oneshot

In [58]:
# Định dạng prompt kiểu hội thoại
def build_prompt(user_input):
    return f"<|user|>\n{user_input}\n<|assistant|>\n"

# Hàm sinh văn bản từ mô hình
def get_completion(prompt):
    return model(prompt).strip()

def translate_and_save(sentences, source_language, target_language, output_file):
    # Prompt for machine translation task
    prompt_intro = (
        "در ادامه‌، وظیفه‌ (تکلیفی) برای شما درنظر گرفته شده است، که مربوط به  ترجمه ماشینی است. "
        "به این منظور جمله‌ای به زبان انگلیسی به شما نشان داده می‌شود. "
        "در نتیجه، شما باید آن جمله را به شکل مناسب و دقیق به زبان فارسی ترجمه کنید."

        " مثالی از ترجمه‌ی جمله‌ی نمونه از زبان انگلیسی به زبان فارسی:"
        " 'This is a sample sentence that we are going to translate it.' -> 'این یک جمله‌ی نمونه است که می‌خواهیم آن را ترجمه کنیم.'"
    )

    # Get the translation for each sentence and save it to a file
    with open(output_file, 'w', encoding='utf-8') as f:
        for sentence in sentences:
            prompt = f"{prompt_intro}\n\nجمله زیر به زبان فارسی است و باید به زبان انگلیسی ترجمه شود:\n {sentence}"

            # Get the translation for the current sentence
            response = get_completion(prompt)

            # Extract and save the translated sentence
            translated_sentence = response.strip()
            f.write(translated_sentence + '\n')

            # Add a delay of 20 seconds between requests
            time.sleep(20)

In [59]:
translate_and_save(
    en_sentences_10,
    source_language="English",
    target_language="Persian",
    output_file="/content/PERPrompt-EN-PER-1shot (openchat).txt"
)

In [60]:
prener1_file_path = "/content/PERPrompt-EN-PER-1shot (openchat).txt"

with open(prener1_file_path, "r") as file:
    prener1 = [line.strip() for line in file]

bleu = sacrebleu.corpus_bleu(prener0, [persian])

print(bleu)

BLEU = 0.12 9.6/0.2/0.1/0.0 (BP = 0.406 ratio = 0.526 hyp_len = 622 ref_len = 1182)
